# Intro y Recursos

---
#### *Análisis de sentimientos en redes sociales para turismo accesible de personas con discapacidad*
creamos un programa que extrae datos de Twitter mediante credenciales de autenticación de Twitter Developer y criterios definidos.
Los resultados se almacenan en un archivo CSV después de aplicar un proceso de normalización para eliminar stopwords, espacios,
caracteres especiales y emojis. El código también identifica palabras concurrentes y relaciones, lo que permite generar una nube de
palabras para visualizar la relevancia de términos específicos. Luego de traducir los resultados al español, se realiza el
preentrenamiento del modelo y se determina la polaridad de los datos extraídos (Positivo, Negativo, Neutral) utilizando el módulo VADER de NLTK.

---
#### References & Resources
##### *Twitter*
* Twitter API - https://developer.twitter.com/en
* Tweepy - http://docs.tweepy.org/en/latest/index.html

# Parte 1: Credenciales de la API de Twitter, autenticación e inicio de la clase "cliente"

In [ ]:
my_consumer_key = "enC7ohO0r7TYhLOIsFcRhLs6G"
my_consumer_secret = "3NcbQ2W8qnpcgn6veRsnWtZr8sUuXR82pjgR5BYf2b6gWPxOLf"

my_access_token = "1716598343055654912-LuPRpYFYpYCwU83EexABetrwjn7NtK"
my_access_secret = "43aLiBO0RkKnqdGPtHo0zqRYuy1f3U7jL8D7ol4E9SxwS"

my_bearer_token = "AAAAAAAAAAAAAAAAAAAAANusqgEAAAAAZCyTycYGJHPr3ri%2BeE3A5bflHzM%3DKbiz0Iy9KGaooiNAan9XgpoP0XdFfHVbkKCke0A4mEkkZTKZyu"

In [ ]:
!pip install git+https://github.com/tweepy/tweepy.git --upgrade

  Cloning https://github.com/tweepy/tweepy.git to /tmp/pip-req-build-jjrch6cs
  Running command git clone --filter=blob:none --quiet https://github.com/tweepy/tweepy.git /tmp/pip-req-build-jjrch6cs
  Resolved https://github.com/tweepy/tweepy.git to commit f32d12dbddbd877470446657812a10a04292d0c9
  Preparing metadata (setup.py) ... done
  Created wheel for tweepy: filename=tweepy-4.14.0-py3-none-any.whl size=99197 sha256=501342abed8c21469f49d1fc952dc35d8bf2d82698dd779e3f88b2a685b1a1be
  Stored in directory: /tmp/pip-ephem-wheel-cache-whm_wyzf/wheels/dc/75/73/ac2b7c1ac66d801a0b03c7707a2fc16e8689f792b585994c6f
Successfully built tweepy
  Attempting uninstall: tweepy
    Found existing installation: tweepy 4.13.0
    Uninstalling tweepy-4.13.0:
      Successfully uninstalled tweepy-4.13.0


In [ ]:
# Import the tweepy package
import tweepy

# Import a variety of other packages that may be useful for working with data.
import pandas as pd
import json
import time

In [ ]:
client = tweepy.Client(
    wait_on_rate_limit = True,
    consumer_key = my_consumer_key,
    consumer_secret = my_consumer_secret,
    access_token = my_access_token,
    access_token_secret = my_access_secret,
    bearer_token = my_bearer_token,
)

In [ ]:
# Test to make sure the client object exists

client

In [ ]:
client.get_user(username ="BarriosJuan20")

Forbidden: ignored

# Parte 2: Primeros pasos con Tweepy y la búsqueda de usuarios

In [ ]:
# Las cuentas de usuario individuales son un buen punto de partida para aprender a
# recuperar y trabajar con datos de la API de Twitter. El siguiente ejemplo utiliza
# nuestro objeto cliente y la función get_user(...) para buscar un objeto específico

user_single = client.get_user(
    username ="barrosjss",
)

In [ ]:
user_single

Response(data=<User id=1009862863849181184 name=Jesús Barros username=barrosjss>, includes={}, errors=[], meta={})

In [ ]:
# Se puede acceder a los datos como componente del objeto user_single

user_single.data

<User id=1009862863849181184 name=Jesús Barros username=barrosjss>

In [ ]:
user_single.data.name

'Jesús Barros'

In [ ]:
print( user_single.data.id )
print( user_single.data.name )
print( user_single.data.username )
print( user_single.data.created_at )

1009862863849181184
Jesús Barros
barrosjss
None


In [ ]:
# Esta información está estructurada como un diccionario y puede tratarse como un JSON

user_single.data.data

{'id': '1009862863849181184', 'name': 'Jesús Barros', 'username': 'barrosjss'}

In [ ]:
# Sin embargo, lo que supongo que es más útil para la mayoría de los investigadores es utilizar Pandas.
# para convertir los datos a un Pandas DataFrame: https://pandas.pydata.org/pandas-docs/stable/user_guide/dsintro.html#dataframe

pd.json_normalize(user_single.data.data, sep="_" )

,id,name,username
0,1009862863849181184,Jesús Barros,barrosjss


In [ ]:
# Muchas funciones en la API de Twitter/Tweepy permiten enviar múltiples
# solicitudes a la vez. En este caso, podemos solicitar detalles de la cuenta en múltiples
# cuentas de Twitter conocidas.

users_group = client.get_users(
    usernames = ["standupmaths","FryRsquared","3blue1brown","MouldS"],
    user_fields = ["created_at", "description","public_metrics","verified"]
)

users_group

Response(data=[<User id=92614042 name=Matt Parker username=standupmaths>, <User id=273375532 name=Hannah Fry username=FryRsquared>, <User id=2877269376 name=Grant Sanderson username=3blue1brown>, <User id=17791581 name=Steve Mould username=MouldS>], includes={}, errors=[], meta={})

In [ ]:
users_group.data

[<User id=92614042 name=Matt Parker username=standupmaths>,
 <User id=273375532 name=Hannah Fry username=FryRsquared>,
 <User id=2877269376 name=Grant Sanderson username=3blue1brown>,
 <User id=17791581 name=Steve Mould username=MouldS>]

In [ ]:
# Crear marco de datos vacío
data_users = pd.DataFrame()

# Iterar a través de users_group y agregar datos al marco de datos data_users
for i in users_group.data:
  temp_data = pd.json_normalize( i.data ,  sep = "_")
  data_users = data_users.append( temp_data , ignore_index=True )

<ipython-input-22-51142f5c634b>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_users = data_users.append( temp_data , ignore_index=True )
<ipython-input-22-51142f5c634b>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_users = data_users.append( temp_data , ignore_index=True )
<ipython-input-22-51142f5c634b>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_users = data_users.append( temp_data , ignore_index=True )
<ipython-input-22-51142f5c634b>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_users = data_users.append( temp_data , ignore_index=True )


In [ ]:
data_users

,verified,name,id,username,description,created_at,public_metrics_followers_count,public_metrics_following_count,public_metrics_tweet_count,public_metrics_listed_count,public_metrics_like_count
0,False,Matt Parker,92614042,standupmaths,"#1 best-selling author, also maths clown.\nVid...",2009-11-25T21:19:33.000Z,138897,619,27245,1408,0
1,True,Hannah Fry,273375532,FryRsquared,All math and no trousers.,2011-03-28T11:14:20.000Z,221921,2129,11530,1600,10847
2,True,Grant Sanderson,2877269376,3blue1brown,Pi creature caretaker.\nContact/faq: https://t...,2014-10-25T20:56:48.000Z,343812,357,3615,2330,1124
3,False,Steve Mould,17791581,MouldS,YouTube: https://t.co/yHFBVVXoMz\nBook me: htt...,2008-12-01T22:37:20.000Z,31874,324,4380,330,375


# Parte 3: Filtrado de twits con palabras claves

In [ ]:
# client = tweepy.Client(
#     wait_on_rate_limit = True,
#     consumer_key = my_consumer_key,
#     consumer_secret = my_consumer_secret,
#     access_token = my_access_token,
#     access_token_secret = my_access_secret,
#     bearer_token = my_bearer_token,
# )

# Post Tweet
#message = " MESSAGE "
#client.create_tweet(text=message)
#print("Tweeted!")


#client.search(q="hola", lang="es", tweet_mode="extended", count=10)

In [ ]:
auth = tweepy.OAuthHandler(my_consumer_key, my_consumer_secret)
auth.set_access_token(my_access_token, my_access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
# Defina los términos de busqueda
search_words = "Shakira"

# Extraiga los tweets
tweets = tweepy.Cursor(api.search_tweets,
                   q=search_words,
                   lang="es",
                   tweet_mode="extended"
                  ).items(10)
#para el resto de parámetros revisar: https://docs.tweepy.org/en/stable/client.html

In [ ]:
tweets_list = [[tweet.full_text, tweet.user.location] for tweet in tweets]
tweets_list

https://www.mincit.gov.co/CMSPages/GetFile.aspx?guid=f29fa89a-ed14-4c7f-b803-176b86074fbc


Forbidden: ignored